In [4]:
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint

# 데이터 분할

In [ ]:
# 랜덤 시드 설정
random.seed(42)

# 데이터셋의 루트 디렉토리와 각 클래스 디렉토리를 지정합니다.
ROOT_DIR = '' # 폴더 경로 지정
CLASS_DIRS = ['normal', 'broken']
NEW_DATA_DIR = 'new split data'

# 각 클래스 디렉토리에 있는 모든 파일명을 읽어들입니다.
data = []
for class_dir in CLASS_DIRS:
    files = os.listdir(os.path.join(ROOT_DIR, class_dir))
    for file in files:
        data.append((os.path.join(ROOT_DIR, class_dir, file), class_dir))

# 데이터를 랜덤하게 train, validation, test로 분할합니다.
train_val_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_val_data, test_size=0.2, random_state=42)

# 'new data' 폴더 생성
os.makedirs(NEW_DATA_DIR, exist_ok=True)

# 분할된 데이터셋을 저장할 디렉토리를 생성합니다.
for dataset in ['train', 'val', 'test']:
    dataset_dir = os.path.join(NEW_DATA_DIR, dataset)
    for class_dir in CLASS_DIRS:
        os.makedirs(os.path.join(dataset_dir, class_dir), exist_ok=True)

# 분할된 데이터셋을 디렉토리에 저장합니다.
for data, dataset in [(train_data, 'train'), (val_data, 'val'), (test_data, 'test')]:
    for file, class_dir in data:
        filename = os.path.basename(file)
        dest_dir = os.path.join(NEW_DATA_DIR, dataset, class_dir)
        dest_file = os.path.join(dest_dir, filename)
        os.makedirs(dest_dir, exist_ok=True)
        os.replace(file, dest_file)

print('Data split completed!')

In [7]:
# 랜덤 시드 설정
random.seed(42)
tf.random.set_seed(42)

# 데이터 폴더 경로
data_dir = './new split data'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

# 이미지 전처리 및 데이터 증강을 위한 설정
image_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

# 훈련 데이터 및 검증 데이터 생성기 설정
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

# 모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 체크포인트 저장 경로 설정
checkpoint_path = './model.ckpt'

# 체크포인트 콜백 생성
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

# 모델 훈련
model.fit(train_generator,
          steps_per_epoch=train_generator.n // batch_size,
          epochs=10,
          validation_data=val_generator,
          validation_steps=val_generator.n // batch_size,
          callbacks=[checkpoint_callback]
         )

# 모델 평가
_, accuracy = model.evaluate(val_generator)
print('Validation accuracy:', accuracy)

Found 2364 images belonging to 2 classes.
Found 592 images belonging to 2 classes.
Epoch 1/10
73/73 [==============================] - ETA: 0s - loss: 0.3273 - accuracy: 0.8641
Epoch 1: val_loss improved from inf to 0.10859, saving model to .\model.ckpt
73/73 [==============================] - 119s 2s/step - loss: 0.3273 - accuracy: 0.8641 - val_loss: 0.1086 - val_accuracy: 0.9427
Epoch 2/10
73/73 [==============================] - ETA: 0s - loss: 0.0804 - accuracy: 0.9717
Epoch 2: val_loss improved from 0.10859 to 0.05293, saving model to .\model.ckpt
73/73 [==============================] - 116s 2s/step - loss: 0.0804 - accuracy: 0.9717 - val_loss: 0.0529 - val_accuracy: 0.9861
Epoch 3/10
73/73 [==============================] - ETA: 0s - loss: 0.0341 - accuracy: 0.9880
Epoch 3: val_loss improved from 0.05293 to 0.02071, saving model to .\model.ckpt
73/73 [==============================] - 115s 2s/step - loss: 0.0341 - accuracy: 0.9880 - val_loss: 0.0207 - val_accuracy: 0.9948
Epoch 